In [10]:
"""
Solving FrozenLake8x8 environment using Policy iteration.
Author : Moustafa Alzantot (malzantot@ucla.edu)
使用策略迭代解决FrozenLake8x8环境。
作者：Moustafa Alzantot（malzantot@ucla.edu）
"""
import numpy as np
import gym
from gym import wrappers


def run_episode(env, policy, gamma = 1.0, render = False):
    """ Runs an episode and return the total reward """
    """运行一集并返回总奖励"""
    obs = env.reset() #初始化环境
    total_reward = 0 #奖励
    step_idx = 0
    while True:
        if render:
            env.render() #图像引擎
        obs, reward, done , _ = env.step(int(policy[obs])) #物理引擎
        total_reward += (gamma ** step_idx * reward)
        step_idx += 1
        if done:
            break
    return total_reward


In [11]:
def evaluate_policy(env, policy, gamma = 1.0, n = 100):  #evaluate评估
    scores = [run_episode(env, policy, gamma, False) for _ in range(n)]
    return np.mean(scores)  #scores 分数

In [12]:
def extract_policy(v, gamma = 1.0):  #extract提取
    """ Extract the policy given a value-function """
    """提取给定值函数的策略"""
    policy = np.zeros(env.nS)
    for s in range(env.nS):
        q_sa = np.zeros(env.nA)
        for a in range(env.nA):
            q_sa[a] = sum([p * (r + gamma * v[s_]) for p, s_, r, _ in  env.P[s][a]])
        policy[s] = np.argmax(q_sa)
    return policy

In [13]:
def compute_policy_v(env, policy, gamma=1.0): #compute计算
    """ Iteratively evaluate the value-function under policy.
    Alternatively, we could formulate a set of linear equations in iterms of v[s] 
    and solve them to find the value function.
        在策略下迭代求值函数。
    或者，我们可以在v[s]的iterms中建立一组线性方程求出它们的值函数。
    """
    v = np.zeros(env.nS)
    eps = 1e-10   #奖励
    while True:
        prev_v = np.copy(v)
        for s in range(env.nS):
            policy_a = policy[s]
            v[s] = sum([p * (r + gamma * prev_v[s_]) for p, s_, r, _ in env.P[s][policy_a]])
        if (np.sum((np.fabs(prev_v - v))) <= eps):  #np.fabs 计算绝对值
            # value converged
            break
    return v

In [14]:
def policy_iteration(env, gamma = 1.0):
    """ Policy-Iteration algorithm """
    """策略迭代算法"""     #choice 随机
    policy = np.random.choice(env.nA, size=(env.nS))  # initialize a random policy初始化随机策略
    max_iterations = 200000
    gamma = 1.0
    for i in range(max_iterations):
        old_policy_v = compute_policy_v(env, policy, gamma)   #计算值函数
        new_policy = extract_policy(old_policy_v, gamma)   #提取策略
        if (np.all(policy == new_policy)):
            print ('Policy-Iteration converged at step %d.' %(i+1))
            break
        policy = new_policy
    return policy

In [ ]:
if __name__ == '__main__':
    env_name  = 'FrozenLake8x8-v0'
    env = gym.make(env_name)
    optimal_policy = policy_iteration(env, gamma = 1.0)
    scores = evaluate_policy(env, optimal_policy, gamma = 1.0)
    print('Average scores = ', np.mean(scores))